# Introduction to Physics-Informed Neural Networks (PINNs)

This notebook provides a comprehensive introduction to Physics-Informed Neural Networks, a revolutionary approach to solving partial differential equations using deep learning.

## Table of Contents
1. [What are PINNs?](#what-are-pinns)
2. [How do PINNs work?](#how-do-pinns-work)
3. [Advantages over traditional methods](#advantages)
4. [Simple example: Solving a 1D ODE](#simple-example)
5. [Key concepts](#key-concepts)

In [ ]:
# Import necessary libraries
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 12

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. What are PINNs? <a id='what-are-pinns'></a>

Physics-Informed Neural Networks (PINNs) are a class of neural networks that are trained to solve supervised learning tasks while respecting any given laws of physics described by general nonlinear partial differential equations.

### Traditional Approach vs PINNs

In [ ]:
# Visualization: Traditional vs PINN approach
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Traditional approach
ax1.set_title('Traditional Numerical Methods', fontsize=16, weight='bold')
ax1.text(0.5, 0.8, 'Discretize Domain', ha='center', fontsize=14, 
         bbox=dict(boxstyle='round,pad=0.5', facecolor='lightcoral'))
ax1.text(0.5, 0.6, '↓', ha='center', fontsize=20)
ax1.text(0.5, 0.4, 'Solve Linear System', ha='center', fontsize=14,
         bbox=dict(boxstyle='round,pad=0.5', facecolor='lightblue'))
ax1.text(0.5, 0.2, '↓', ha='center', fontsize=20)
ax1.text(0.5, 0.0, 'Solution at Grid Points', ha='center', fontsize=14,
         bbox=dict(boxstyle='round,pad=0.5', facecolor='lightgreen'))
ax1.axis('off')

# PINN approach
ax2.set_title('Physics-Informed Neural Networks', fontsize=16, weight='bold')
ax2.text(0.5, 0.8, 'Neural Network', ha='center', fontsize=14,
         bbox=dict(boxstyle='round,pad=0.5', facecolor='lightcoral'))
ax2.text(0.5, 0.6, '↓', ha='center', fontsize=20)
ax2.text(0.5, 0.4, 'Physics Loss Function', ha='center', fontsize=14,
         bbox=dict(boxstyle='round,pad=0.5', facecolor='lightblue'))
ax2.text(0.5, 0.2, '↓', ha='center', fontsize=20)
ax2.text(0.5, 0.0, 'Continuous Solution', ha='center', fontsize=14,
         bbox=dict(boxstyle='round,pad=0.5', facecolor='lightgreen'))
ax2.axis('off')

plt.tight_layout()
plt.show()

## 2. How do PINNs work? <a id='how-do-pinns-work'></a>

The key innovation of PINNs is the physics-informed loss function. Instead of requiring labeled data, PINNs use the PDE itself as the "label".

In [ ]:
# Simple neural network architecture
class SimplePINN(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=50, output_dim=1, num_layers=3):
        super(SimplePINN, self).__init__()
        
        layers = []
        layers.append(nn.Linear(input_dim, hidden_dim))
        layers.append(nn.Tanh())
        
        for _ in range(num_layers - 2):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.Tanh())
        
        layers.append(nn.Linear(hidden_dim, output_dim))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

# Visualize the architecture
model = SimplePINN()
print("PINN Architecture:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters())}")

### The Physics-Informed Loss Function

The total loss function for a PINN consists of:

$$\mathcal{L} = \mathcal{L}_{PDE} + \mathcal{L}_{BC} + \mathcal{L}_{IC} + \mathcal{L}_{data}$$

Where:
- $\mathcal{L}_{PDE}$: PDE residual loss
- $\mathcal{L}_{BC}$: Boundary condition loss
- $\mathcal{L}_{IC}$: Initial condition loss
- $\mathcal{L}_{data}$: Data fitting loss (if available)

In [ ]:
# Demonstrate automatic differentiation
def demonstrate_autograd():
    # Create a simple function: f(x) = x^2
    x = torch.tensor([2.0], requires_grad=True)
    y = x**2
    
    # Compute derivative: df/dx = 2x
    y.backward()
    
    print(f"x = {x.item():.2f}")
    print(f"f(x) = x² = {y.item():.2f}")
    print(f"df/dx = 2x = {x.grad.item():.2f}")
    print(f"\nExpected: df/dx = 2 × {x.item():.2f} = {2 * x.item():.2f}")

demonstrate_autograd()

## 3. Advantages over Traditional Methods <a id='advantages'></a>

PINNs offer several advantages over traditional numerical methods:

In [ ]:
# Create comparison table
import pandas as pd

comparison_data = {
    'Aspect': ['Domain', 'Solution Type', 'Mesh Required', 'Irregular Geometries', 
               'High Dimensions', 'Inverse Problems', 'Noisy Data'],
    'Traditional Methods': ['Discrete Grid', 'Point Values', 'Yes', 'Challenging', 
                           'Computationally Expensive', 'Difficult', 'Sensitive'],
    'PINNs': ['Continuous', 'Continuous Function', 'No', 'Natural', 
              'Feasible', 'Natural', 'Robust']
}

df = pd.DataFrame(comparison_data)
print("Comparison: Traditional Methods vs PINNs")
print("=" * 70)
print(df.to_string(index=False))

## 4. Simple Example: Solving a 1D ODE <a id='simple-example'></a>

Let's solve a simple ordinary differential equation (ODE) to demonstrate PINNs:

$$\frac{du}{dx} + u = 1, \quad x \in [0, 1]$$
$$u(0) = 0$$

The analytical solution is: $u(x) = 1 - e^{-x}$

In [ ]:
# Define the PINN for ODE
class ODE_PINN(nn.Module):
    def __init__(self):
        super(ODE_PINN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 20),
            nn.Tanh(),
            nn.Linear(20, 20),
            nn.Tanh(),
            nn.Linear(20, 1)
        )
    
    def forward(self, x):
        return self.net(x)

# Physics-informed loss
def ode_loss(model, x):
    x.requires_grad = True
    u = model(x)
    
    # Compute du/dx using autograd
    du_dx = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), 
                                create_graph=True)[0]
    
    # ODE residual: du/dx + u - 1 = 0
    residual = du_dx + u - 1
    
    # Boundary condition: u(0) = 0
    x_bc = torch.tensor([[0.0]], requires_grad=True)
    u_bc = model(x_bc)
    
    # Total loss
    loss = torch.mean(residual**2) + (u_bc**2)
    
    return loss

In [ ]:
# Train the PINN
model = ODE_PINN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training data (collocation points)
x_train = torch.rand(100, 1) * 1.0  # Random points in [0, 1]

# Training loop
losses = []
for epoch in range(2000):
    optimizer.zero_grad()
    loss = ode_loss(model, x_train)
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if epoch % 400 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.6f}")

In [ ]:
# Visualize results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Loss history
ax1.semilogy(losses)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Loss')
ax1.grid(True)

# Solution comparison
x_test = torch.linspace(0, 1, 100).reshape(-1, 1)
with torch.no_grad():
    u_pred = model(x_test).numpy()

x_np = x_test.numpy()
u_true = 1 - np.exp(-x_np)

ax2.plot(x_np, u_true, 'k--', linewidth=2, label='Analytical')
ax2.plot(x_np, u_pred, 'r-', linewidth=2, label='PINN')
ax2.set_xlabel('x')
ax2.set_ylabel('u(x)')
ax2.set_title('ODE Solution')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

# Compute error
error = np.mean(np.abs(u_pred - u_true))
print(f"\nMean Absolute Error: {error:.6f}")

## 5. Key Concepts <a id='key-concepts'></a>

### 5.1 Automatic Differentiation

The cornerstone of PINNs is automatic differentiation, which allows us to compute exact derivatives of the neural network output.

In [ ]:
# Demonstrate higher-order derivatives
def demonstrate_higher_order_derivatives():
    x = torch.tensor([1.0], requires_grad=True)
    
    # f(x) = x^3
    y = x**3
    
    # First derivative: f'(x) = 3x^2
    dy_dx = torch.autograd.grad(y, x, create_graph=True)[0]
    
    # Second derivative: f''(x) = 6x
    d2y_dx2 = torch.autograd.grad(dy_dx, x, create_graph=True)[0]
    
    print("Function: f(x) = x³")
    print(f"x = {x.item():.2f}")
    print(f"f(x) = {y.item():.2f}")
    print(f"f'(x) = {dy_dx.item():.2f} (expected: 3x² = {3 * x.item()**2:.2f})")
    print(f"f''(x) = {d2y_dx2.item():.2f} (expected: 6x = {6 * x.item():.2f})")

demonstrate_higher_order_derivatives()

### 5.2 Collocation Points

PINNs use collocation points - randomly sampled points in the domain where the PDE residual is minimized.

In [ ]:
# Visualize collocation points
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# 1D collocation points
x_1d = torch.rand(50) * 2 - 1  # [-1, 1]
ax1.scatter(x_1d, torch.zeros_like(x_1d), alpha=0.6, s=50)
ax1.set_xlim(-1.1, 1.1)
ax1.set_ylim(-0.5, 0.5)
ax1.set_xlabel('x')
ax1.set_title('1D Collocation Points')
ax1.grid(True)

# 2D collocation points
n_points = 200
x_2d = torch.rand(n_points) * 2 - 1
y_2d = torch.rand(n_points) * 2 - 1
ax2.scatter(x_2d, y_2d, alpha=0.6, s=30)
ax2.set_xlim(-1.1, 1.1)
ax2.set_ylim(-1.1, 1.1)
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_title('2D Collocation Points')
ax2.grid(True)
ax2.set_aspect('equal')

plt.tight_layout()
plt.show()

### 5.3 Loss Weighting

Different components of the loss function may need different weights for optimal training:

In [ ]:
# Demonstrate the effect of loss weighting
def weighted_loss_example():
    # Example loss components
    epochs = np.arange(100)
    
    # Different weighting schemes
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    weights = [
        {'pde': 1.0, 'bc': 1.0, 'ic': 1.0},
        {'pde': 1.0, 'bc': 10.0, 'ic': 10.0},
        {'pde': 0.1, 'bc': 50.0, 'ic': 50.0}
    ]
    
    for idx, w in enumerate(weights):
        # Simulate loss components
        pde_loss = 0.1 * np.exp(-epochs/20) + 0.001
        bc_loss = 0.5 * np.exp(-epochs/15) + 0.0001
        ic_loss = 0.3 * np.exp(-epochs/10) + 0.0001
        
        total_loss = w['pde']*pde_loss + w['bc']*bc_loss + w['ic']*ic_loss
        
        ax = axes[idx]
        ax.semilogy(epochs, total_loss, 'k-', linewidth=2, label='Total')
        ax.semilogy(epochs, pde_loss, 'r--', label='PDE')
        ax.semilogy(epochs, bc_loss, 'g--', label='BC')
        ax.semilogy(epochs, ic_loss, 'b--', label='IC')
        
        ax.set_xlabel('Epoch')
        ax.set_ylabel('Loss')
        ax.set_title(f"Weights: PDE={w['pde']}, BC={w['bc']}, IC={w['ic']}")
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

weighted_loss_example()

## Summary

In this notebook, we've covered:

1. **What PINNs are**: Neural networks that learn to satisfy PDEs
2. **How they work**: Using physics-informed loss functions and automatic differentiation
3. **Their advantages**: Continuous solutions, mesh-free, handling complex geometries
4. **A simple example**: Solving an ODE with PINNs
5. **Key concepts**: Automatic differentiation, collocation points, and loss weighting

### Next Steps

In the next notebook, we'll apply these concepts to solve the wave equation, a more complex partial differential equation that describes wave propagation.

## Exercises

1. **Modify the ODE example** to solve: $\frac{du}{dx} - 2u = -4, u(0) = 1$
   - Hint: The analytical solution is $u(x) = 2 - e^{2x}$

2. **Experiment with network architecture**: Try different numbers of layers and neurons

3. **Investigate collocation points**: How does the number of collocation points affect accuracy?

4. **Loss weighting**: For the ODE example, what happens if you weight the boundary condition loss differently?

In [ ]:
# Space for exercises
